In [1]:
import os
import pickle
import pandas as pd
from nltk.metrics import *
from nltk.tokenize import TweetTokenizer
import string
from nltk.corpus import stopwords
import nltk
from collections import defaultdict

In [2]:
# Get current path of directory
dir_path = os.getcwd()

In [3]:
# Data extration

df = pd.read_csv(dir_path + '/GOP_REL_ONLY.csv', encoding='ISO-8859-1')
to_drop = ['candidate','candidate:confidence','relevant_yn','relevant_yn:confidence',
'sentiment:confidence','subject_matter','subject_matter:confidence','candidate_gold',
'name','relevant_yn_gold','retweet_count','sentiment_gold','subject_matter_gold',
'tweet_coord','tweet_created','tweet_id','tweet_location','user_timezone' ]
df.drop(to_drop, axis=1)


,sentiment,text
0,Neutral,RT @NancyLeeGrahn: How did everyone feel about...
1,Positive,RT @ScottWalker: Didn't catch the full #GOPdeb...
2,Neutral,RT @TJMShow: No mention of Tamir Rice and the ...
3,Positive,RT @RobGeorge: That Carly Fiorina is trending ...
4,Positive,RT @DanScavino: #GOPDebate w/ @realDonaldTrump...
5,Positive,"RT @GregAbbott_TX: @TedCruz: ""On my first day ..."
6,Negative,RT @warriorwoman91: I liked her and was happy ...
7,Neutral,Going on #MSNBC Live with @ThomasARoberts arou...
8,Negative,Deer in the headlights RT @lizzwinstead: Ben C...
9,Negative,RT @NancyOsborne180: Last night's debate prove...


In [4]:
from nltk.corpus import twitter_samples
twitter_samples.fileids()

pos = twitter_samples.strings('positive_tweets.json')
neg = twitter_samples.strings('negative_tweets.json')
pos_df = pd.DataFrame(data=[tweet for tweet in pos], columns=['text'])
pos_df['sentiment'] = 'Positive'
neg_df = pd.DataFrame(data=[tweet for tweet in neg], columns=['text'])
neg_df['sentiment'] = 'Negative'

len(df)

13871

In [5]:
frames = [pos_df, neg_df, df]
df = pd.concat(frames)
print(len(df))

23871


In [6]:
# Save model to pickle file
def save_pickle(classifier, name):
    save_classifier = open(name + ".pickle", 'wb')
    pickle.dump(classifier, save_classifier)
    save_classifier.close()

In [7]:
tweets = df
len(tweets)

23871

In [8]:
import re, nltk
import string
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()

    
punctuation = list(string.punctuation)
stop = stopwords.words('english') + punctuation + ['rt', 'via', 'RT', ':', ',', '’',
                                                   'i\'m', '…', '...', 'a', 'b', 'c'
                                                  'd','g','j','m','p','s','v','y',
                                                  'e','h','k','n','q','t','w','z',
                                                  'f','i','l','o','r','u','x', 'http', 'https'] 

def normalizer(tweet):
    only_letters = re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z\:\-\)\(])|(\w+:\/\/\S+)", " ", tweet) 
    tokens = TweetTokenizer().tokenize(only_letters)
    lower_case = [l.lower() for l in tokens]
    filtered_result = list(filter(lambda l: l not in stop, lower_case))
    lemmas = [wordnet_lemmatizer.lemmatize(t) for t in filtered_result]
    return lemmas

In [9]:
pd.set_option('display.max_colwidth', -1) # Setting this so we can see the full content of cells
tweets = df[df.sentiment != 'Neutral']
tweets['normalized_tweet'] = tweets.text.apply(normalizer)
tweets[['text','normalized_tweet']].head()


C:\Users\blahb\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,text,normalized_tweet
0,#FollowFriday @France_Inte @PKuchly57 @Milipol_Paris for being top engaged members in my community this week :),"[followfriday, inte, paris, top, engaged, member, community, week, :)]"
1,@Lamb2ja Hey James! How odd :/ Please call our Contact Centre on 02392441234 and we will be able to assist you :) Many thanks!,"[hey, james, odd, please, call, contact, centre, 02392441234, able, assist, :), many, thanks]"
2,@DespiteOfficial we had a listen last night :) As You Bleed is an amazing track. When are you in Scotland?!,"[listen, last, night, :), bleed, amazing, track, scotland]"
3,@97sides CONGRATS :),"[congrats, :)]"
4,yeaaaah yippppy!!! my accnt verified rqst has succeed got a blue tick mark on my fb profile :) in 15 days,"[yeaaaah, yippppy, accnt, verified, rqst, succeed, got, blue, tick, mark, fb, profile, :), 15, day]"


In [10]:
from nltk import ngrams
def ngrams(input_list):
    #onegrams = input_list
    bigrams = [' '.join(t) for t in list(zip(input_list, input_list[1:]))]
    trigrams = [' '.join(t) for t in list(zip(input_list, input_list[1:], input_list[2:]))]
    return bigrams+trigrams
tweets['grams'] = tweets.normalized_tweet.apply(ngrams)
tweets[['grams']].head()

C:\Users\blahb\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


,grams
0,"[followfriday inte, inte paris, paris top, top engaged, engaged member, member community, community week, week :), followfriday inte paris, inte paris top, paris top engaged, top engaged member, engaged member community, member community week, community week :)]"
1,"[hey james, james odd, odd please, please call, call contact, contact centre, centre 02392441234, 02392441234 able, able assist, assist :), :) many, many thanks, hey james odd, james odd please, odd please call, please call contact, call contact centre, contact centre 02392441234, centre 02392441234 able, 02392441234 able assist, able assist :), assist :) many, :) many thanks]"
2,"[listen last, last night, night :), :) bleed, bleed amazing, amazing track, track scotland, listen last night, last night :), night :) bleed, :) bleed amazing, bleed amazing track, amazing track scotland]"
3,[congrats :)]
4,"[yeaaaah yippppy, yippppy accnt, accnt verified, verified rqst, rqst succeed, succeed got, got blue, blue tick, tick mark, mark fb, fb profile, profile :), :) 15, 15 day, yeaaaah yippppy accnt, yippppy accnt verified, accnt verified rqst, verified rqst succeed, rqst succeed got, succeed got blue, got blue tick, blue tick mark, tick mark fb, mark fb profile, fb profile :), profile :) 15, :) 15 day]"


In [11]:
import collections
def count_words(input):
    cnt = collections.Counter()
    for row in input:
        for word in row:
            cnt[word] += 1
    return cnt

In [12]:
tweets[(tweets.sentiment == 'Positive')][['grams']].apply(count_words)['grams'].most_common(20)


[('gopdebate gopdebates', 231),
 ('last night', 175),
 ('lt 3', 135),
 ('fox news', 120),
 ('follow back', 84),
 ('back :)', 82),
 ('looking forward', 80),
 ('rubio gopdebate', 75),
 ('rating gopdebate', 74),
 ('cruz trump', 74),
 ('get rid', 73),
 ('job get', 73),
 ('thanks :)', 73),
 ('expose set', 72),
 ('bush rubio gopdebate', 72),
 ('rid bush rubio', 72),
 ('bush rubio', 72),
 ('set job get', 72),
 ('together expose set', 72),
 ('trump need band', 72)]

In [13]:
tweets[(tweets.sentiment == 'Negative')][['grams']].apply(count_words)['grams'].most_common(20)


[('gopdebate gopdebates', 1051),
 ('fox news', 636),
 ('last night', 419),
 ('jeb bush', 290),
 ('chris wallace', 266),
 ('donald trump', 245),
 ('debate gopdebate', 226),
 ('fair amp', 204),
 ('amp balanced', 202),
 ('fair amp balanced', 202),
 ('debate gopdebate gopdebates', 168),
 ('listen gopdebate', 165),
 ('bush reminds', 163),
 ('music hear', 162),
 ('listen gopdebate gopdebates', 162),
 ('reminds elevator', 162),
 ('hear listen gopdebate', 162),
 ('jeb bush reminds', 162),
 ('music hear listen', 162),
 ('hear listen', 162)]

In [57]:
import numpy as np
from scipy.sparse import hstack
from sklearn.feature_extraction.text import CountVectorizer

# Feature Extraction, build a dictionary of feature indices
# Convert a collection of text documents to a matrix of token counts
count_vectorizer = CountVectorizer(ngram_range=(1,2))

# Learn the vocabulary dictionary and return term-document matrix.
# Fitting finds the internal parameters of the model
# Transform applies those parameters to data
vectorized_data = count_vectorizer.fit_transform(tweets.text)

# Stack horizontally, remove non-zero feature vectors
indexed_data = hstack((np.array(range(0,vectorized_data.shape[0]))[:,None], vectorized_data))

In [15]:
def sentiment2target(sentiment):
    return {
        'Negative': 0,
        'Positive' : 1
    }[sentiment]

# Convert Classification into integer for speed and space efficiency reasons
targets = tweets.sentiment.apply(sentiment2target)

In [16]:
# Split data into training and testing

from sklearn.model_selection import train_test_split
data_train, data_test, targets_train, targets_test = train_test_split(indexed_data, targets, test_size=0.25, random_state=0)
data_train = data_train[:,1:]
data_test = data_test[:,1:]

In [17]:
from sklearn import svm

clf = svm.SVC(gamma=0.01, C=100., probability=True, class_weight='balanced', kernel='linear')
clf_output = clf.fit(data_train, targets_train)

In [18]:
clf.score(data_test, targets_test)

0.7999228246189466

In [19]:
save_pickle(clf, "svm")

In [29]:
from sklearn.naive_bayes import MultinomialNB
ddd = MultinomialNB().fit(data_train, targets_train)

In [30]:
ddd.score(data_test, targets_test)

0.779471348639784

In [22]:
predicted = clf.predict(data_test)

np.mean(predicted == targets_test)

0.7999228246189466

In [23]:
from sklearn import metrics
print(metrics.classification_report(targets_test, predicted, target_names=tweets.sentiment))

             precision    recall  f1-score   support

   Positive       0.83      0.87      0.85      3395
   Positive       0.73      0.66      0.70      1788

avg / total       0.80      0.80      0.80      5183



C:\Users\blahb\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1428: UserWarning: labels size, 2, does not match size of target_names, 20729
  .format(len(labels), len(target_names))


In [24]:
sentiment_counts = tweets.sentiment.value_counts()
num_of_tweets = tweets.text.count()
print(sentiment_counts)


Negative    13493
Positive    7236 
Name: sentiment, dtype: int64


In [25]:
from sklearn.pipeline import Pipeline

vec_clf = Pipeline([('vectorizer', count_vectorizer), ('svm', clf)])
vec_clf.fit(tweets.text, targets)

Pipeline(memory=None,
     steps=[('vectorizer', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 2), preprocessor=None, stop_words=None,
       ...',
  max_iter=-1, probability=True, random_state=None, shrinking=True,
  tol=0.001, verbose=False))])

In [26]:
predicted = vec_clf.predict(tweets.text)

In [27]:
np.mean(predicted == targets)

0.9876501519610208

In [28]:
save_pickle(vec_clf,'vec_clf')